In [17]:
import laspy
import glob
from pathlib import Path

In [18]:
files = glob.glob("data/*.xyz_label_conf")
files[0]

'data/oakland_part3_ao.xyz_label_conf'

In [3]:
new_file = Path(files[0].split('.')[0]+'.las')

In [4]:
outfile_temp = Path.cwd().joinpath("data/tmp/" + str(new_file.name))
outfile_hdr = laspy.header.Header()
outfile = laspy.file.File(outfile_temp, mode='w', header=outfile_hdr)

FileNotFoundError: [Errno 2] No such file or directory: '/home/chase/PycharmProjects/PointCloud/data/tmp/oakland_part3_ao.las'

In [5]:
import pandas as pd
df = pd.read_csv(files[0], 
                 names=['x','y','z','label','conf'],
                 delimiter = ' ',
                 skiprows=63)

In [6]:
outfile.header.max = [max(df['x']), max(df['y']), max(df['z'])]
minx = min(df['x'])
miny = min(df['y'])
minz = min(df['z'])
outfile.header.min = [minx, miny, minz]
outfile.header.offset = [0., 0., 0.]
outfile.header.scale = [.01, .01, .01]  

NameError: name 'outfile' is not defined

In [7]:
df['x'] = df['x']*100
df['y'] = df['y']*100
df['z'] = df['z']*100

In [8]:
df = df.astype('int64')

In [9]:
outfile.X = df['x']
outfile.Y = df['y']
outfile.Z = df['z']

NameError: name 'outfile' is not defined

In [10]:
outfile.x

NameError: name 'outfile' is not defined

In [11]:
new_file.parent.joinpath('las').joinpath(new_file.name)

PosixPath('data/las/oakland_part3_ao.las')

In [12]:
outfile.x

NameError: name 'outfile' is not defined

In [20]:
files = glob.glob("data/*.xyz_label_conf")
for file in files[1:]:
    print(file)
    new_file = Path(file.split('.')[0]+'.las')
    outfile_temp = Path.cwd().joinpath("data/las/" + str(new_file.name))
    outfile_hdr = laspy.header.Header()
    outfile = laspy.file.File(outfile_temp, mode='w', header=outfile_hdr)
    df = pd.read_csv(file, 
                 names=['x','y','z','label','conf'],
                 delimiter = ' ',
                 skiprows=63)
    outfile.header.max = [max(df['x']), max(df['y']), max(df['z'])]
    minx = min(df['x'])
    miny = min(df['y'])
    minz = min(df['z'])
    outfile.header.min = [minx, miny, minz]
    outfile.header.offset = [0., 0., 0.]
    outfile.header.scale = [.01, .01, .01] 
    df['x'] = df['x']*100
    df['y'] = df['y']*100
    df['z'] = df['z']*100
    df = df.astype('int64')
    outfile.X = df['x']
    outfile.Y = df['y']
    outfile.Z = df['z']
    outfile.close()

data/oakland_part3_am.xyz_label_conf
data/oakland_part2_ac.xyz_label_conf
data/oakland_part2_al.xyz_label_conf
data/oakland_part3_ap.xyz_label_conf
data/oakland_part2_ak.xyz_label_conf
data/oakland_part2_ad.xyz_label_conf
data/oakland_part2_ai.xyz_label_conf
data/oakland_part2_ag.xyz_label_conf
data/oakland_part2_aj.xyz_label_conf
data/oakland_part2_ae.xyz_label_conf
data/oakland_part3_ak.xyz_label_conf
data/oakland_part2_ah.xyz_label_conf
data/oakland_part3_an.xyz_label_conf
data/oakland_part2_ao.xyz_label_conf


In [ ]:
len(files)